In [ ]:
import annotation.tailrec

Scala- Folds
=========

##### Vassil Dichev

24.03.2021

# Type parameters

In [ ]:
@tailrec
def map(l: List[Int], f: Int => Int, acc: List[Int] = Nil): List[Int] =
  if (l.isEmpty) acc.reverse
  else map(l.tail, f, l.head :: acc)

In [36]:
@tailrec
def map[A,B](l: List[A], f: A => B, acc: List[B] = Nil): List[B] =
  if (l.isEmpty) acc.reverse
  else map(l.tail, f, l.head :: acc)

cmd36.sc:4: type mismatch;
 found   : rassoc$1.type (with underlying type A)
 required: B
  else map(l.tail, f, l.head :: acc)
                        ^Compilation Failed

: 

# reduce
![image](images/reduce.png)

# Folding right or left
![image](images/Right-fold-transformation.png)

![image](images/Left-fold-transformation.png)


In [38]:
def map[A,B](l: List[A])(f: A => B): List[B] = l.foldRight(List.empty[B]){(x, acc) => f(x) :: acc}
map(List(1, 2, 3))(_ * 2)

defined function map
res37_1: List[Int] = List(2, 4, 6)

In [0]:
def map[A,B](l: List[A])(f: A => B): List[B] =
  l.foldRight(List.empty[B]) {
    (x, l) => f(x) :: l
  }

In [41]:
def filter[A](l: List[A])(f: A => Boolean): List[A] = l.foldRight(List.empty[A]){(x, acc) => if (f(x)) x :: acc else acc}
filter(List(2, 3, 4))(_ % 2 == 0)
filter(List.empty[Int])(_ % 2 == 0)

defined function filter
res40_1: List[Int] = List(2, 4)
res40_2: List[Int] = List()

In [0]:
def filter[A](l: List[A])(f: A => Boolean): List[A] =
  l.foldRight(List.empty[A]) {
    (x, l) => if (f(x)) x :: l else l
  }

# Exercise: implementing in terms of foldXXX
1. We'll guess the function from the signature
2. We'll let the signature guide the implementation

In [ ]:

def f1[A](l: List[A]): Int = ???

In [45]:
def size[A](l: List[A]): Int = l.foldLeft(0) {(acc, _) => acc + 1 }
size(List(1, 2, 3))

defined function size
res44_1: Int = 3

In [ ]:
def size[A](l: List[A]) = l.foldLeft(0)((acc, _) => 1 + acc)

In [ ]:
def f2(l: List[Int]): Int = ???

In [49]:
def max(l: List[Int]): Int = l.reduceLeft(math.max)
max(Nil)

: 

In [ ]:
def max(l: List[Int]) = l reduce math.max

In [ ]:
def f3[A](l: List[A]): List[A] = ???

In [52]:
def reverse[A](l: List[A]): List[A] = l.foldLeft(List.empty[A])((acc, x) => x :: acc)
reverse(List(1, 2, 3))

defined function reverse
res51_1: List[Int] = List(3, 2, 1)

In [ ]:
def reverse[A](l: List[A]) = l.foldLeft(List.empty[A])((acc, x) => x :: acc)

# reverse
![image](images/animation-reverse.gif)

In [ ]:
def f4[A](l1: List[A], l2: List[A]): List[A] = ???

In [56]:
def concat[A](l1: List[A], l2: List[A]): List[A] = l1.foldRight(l2){ _ :: _ }
concat(List(1, 5, 3), List(4, 2, 6))

defined function concat
res55_1: List[Int] = List(1, 5, 3, 4, 2, 6)

In [ ]:
def concat[A](l1: List[A], l2: List[A]) = l1.foldRight(l2) { _ :: _ }

In [ ]:
def f5[A](l: List[A], x: A): Boolean = ???

In [ ]:
def contains[A](l: List[A], el: A): Boolean = ???

In [ ]:
def contains[A](l: List[A], el: A) = l.foldLeft(false)((acc, x) => x == el || acc)

# Lazy
![image](images/work.png)


#  Fold and laziness



In [0]:
def forall[A](l: List[A])(f: A => Boolean): Boolean = ???

#  Fold and laziness



In [0]:
def forall[A](l: List[A])(f: A => Boolean): Boolean =
  l.foldLeft(true) { (acc, x) => f(x) && acc }

In [ ]:
def forall[A](l: List[A])(f: A => Boolean): Boolean = {
  l.foldLeft(true) { (acc, x) =>
    println("Executing for " + x)
    f(x) && acc
  }
}

forall(List.range(1, 5))(_ % 2 == 0)

#  forall/exists



In [0]:
def exists[A](l: List[A])(f: A => Boolean): Boolean = ???

#  forall/exists



In [0]:
def exists[A](l: List[A])(f: A => Boolean): Boolean =
  !forall(l)(!f(_))

# Case study: side effects

## Migration of Scalaz to Cats

### Scalaz' implementation:
```
l.reverse.foldLeft(...) {...}
```

### Cats' implementation:
```
l.foldRight(...) {...}
```

Scala- Collections
=========

##### Vassil Dichev

31.03.2021

# Recap


## What is the type of a function?

## How do partially applied and curried functions compare?

## How are multiple parameter lists and currying related?

## What is the difference between a function and its tupled version?

# Collections
![image](images/collections-diagram-213.svg)


#  Design tips

* Prefer `var` of immutable to `val` of mutable



In [0]:
var map = Map.empty[Int,String]

In [0]:
val map = collection.mutable.Map.empty[Int,String]

# Collections are functions



In [0]:
val dict = Map(1 -> "one", 2 -> "two")
List(1, 2) map dict

In [0]:
val arr = Array(5, 3, 4)
List(2, 1) map arr

In [0]:
val set = Set(1, 3, 5, 8)
(1 to 10) map set

# Uniform return type principle



In [0]:
arr map (_ + 2)

In [0]:
dict map (t => t._1.toString -> t._2)

In [0]:
import Function.tupled
dict map tupled(_.toString -> _)

In [0]:
dict.mapValues("<" + _ + ">")

In [0]:
set map (_ / 2)

# Partial functions


In [0]:
val reciprocal = new PartialFunction[Int,Double] {
  override def isDefinedAt(x: Int) = x != 0
  override def apply(x: Int) =
    if (isDefinedAt(x)) 1.toDouble / x
    else throw new MatchError("Division by zero")
}

In [ ]:
val reciprocal: PartialFunction[Int,Double] = {
  case x if x != 0 => 1.toDouble / x
}

# Tuples



In [0]:
val t1: (Int, String) = (1, "hi")

In [0]:
val t2: Tuple2[Int,String] = Tuple2(1, "hi")

In [0]:
val t3 = 1 -> "hi"

In [0]:
val a = t1._1
val b = t3._2

# Tuple2

-   `partition`- combines `filter`/`filterNot`

-   `span`- combines `takeWhile`/`dropWhile`

-   `splitAt`- combines `take`/`drop`

# Functions

-   `flatten`

-   `flatMap`

-   `groupBy`

-   `zip`

# flatten/flatMap

Generate all combinations of numbers



In [0]:
val s = "abcd"
val prefixes = s.inits.toList
val suffixes = s.tails.toList

In [0]:
val lol = prefixes.map(_.tails.toList)

In [0]:
lol.flatten

In [0]:
prefixes.flatMap(_.tails)

In [0]:
lol.flatMap(identity) == lol.flatten

In [ ]:
val list = List(1, 2, 3)
for {
  i <- list
  if i % 2 == 0
  c <- 'a' to 'c'
} yield s"$i$c"

In [ ]:
val list = List(1, 2, 3)

list.filter(i => i % 2 != 0).flatMap(i => ('a' to 'c').map(c => s"$i$c"))

# Syntax sugar
![image](images/godji-opakovka2.jpg)

# Merging maps



In [0]:
val eng = Map(1 -> "one", 2 -> "two")
val ger = Map(1 -> "eins", 3 -> "drei")

In [0]:
List(eng, ger).flatten.groupBy(_._1).mapValues(_.map(_._2))

# Zip
![image](images/zipper.png)


# Exercise

Find the differences between consecutive numbers



In [0]:
val l = List(1, 3, 8, 11, 15, 17, 24, 27, 32)

In [0]:
(l zip l.tail)

In [0]:
(l.tail zip l) map tupled(_ - _)

# Laziness

> I'm not lazy, I'm just on energy-saving mode.
>
> **Anonymous**

* non-strict: arguments are evaluated on demand
* lazy: evaluation is memoized

In [ ]:
def doubleView(s: Seq[Int]) =
  s.view.map{ x => println("Calculating " + x); x * 2 }
{{
  val longRange = 1 to 10000
  val view = doubleView(longRange).take(10)
  println(view.toList)
  println(view.toList)
}}

In [ ]:
def doubleLazyList(s: Seq[Int]) =
  s.to(LazyList).map{ x => println("Calculating " + x); x * 2 }
{{
  val longRange = 1 to 10000
  val stream = doubleLazyList(longRange).take(10)
  println(stream.toList)
  println(stream.toList)
}}

In [ ]:
{{
  lazy val fibs: LazyList[Int] = 0 #:: 1 #:: fibs.zip(fibs.tail).map(n => n._1 + n._2 )
  fibs.take(5)
}}

# Let's implement a Lazy List!

In [ ]:
trait LazyIntList {
  def head: Int
  def tail: LazyIntList
}
case object Nil extends LazyIntList {
  def head = throw new NoSuchElementException
  def tail = throw new UnsupportedOperationException
}
case class LazyCons(head: Int, tail: LazyIntList) extends LazyIntList


In [ ]:
trait LazyIntList {
  def head: Int
  def tail: LazyIntList
}

In [ ]:
object LazyNil extends LazyIntList {
  def head = throw new NoSuchElementException
  def tail = throw new UnsupportedOperationException
}

In [ ]:
class LazyCons(h: => Int, t: => LazyIntList) extends LazyIntList {
  lazy val head = h
  lazy val tail = t
}

In [ ]:
def from(start: Int, step: Int = 1): LazyIntList = new LazyCons(start, from(start + step, step))
val naturalNumbers = from(0)